In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('DatafinitiElectronicsProductsPricingData.csv')
df.head()

,id,prices.amountMax,prices.amountMin,prices.availability,prices.condition,prices.currency,prices.dateSeen,prices.isSale,prices.merchant,prices.shipping,...,name,primaryCategories,sourceURLs,upc,weight,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30
0,AVphzgbJLJeJML43fA0o,104.99,104.99,Yes,New,USD,"2017-03-30T06:00:00Z,2017-03-10T22:00:00Z,2017...",False,Bestbuy.com,NaN,...,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Electronics,https://www.amazon.com/Sanus-VLF410B1-10-Inch-...,7.93796E+11,32.8 pounds,NaN,NaN,NaN,NaN,NaN
1,AVpgMuGwLJeJML43KY_c,69.00,64.99,In Stock,New,USD,2017-12-14T06:00:00Z,True,Walmart.com,Expedited,...,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds,NaN,NaN,NaN,NaN,NaN
2,AVpgMuGwLJeJML43KY_c,69.00,69.00,In Stock,New,USD,2017-09-08T05:00:00Z,False,Walmart.com,Expedited,...,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds,NaN,NaN,NaN,NaN,NaN
3,AVpgMuGwLJeJML43KY_c,69.99,69.99,Yes,New,USD,2017-10-10T05:00:00Z,False,Bestbuy.com,NaN,...,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds,NaN,NaN,NaN,NaN,NaN
4,AVpgMuGwLJeJML43KY_c,66.99,66.99,Yes,New,USD,2017-08-28T07:00:00Z,False,Bestbuy.com,NaN,...,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Electronics,http://reviews.bestbuy.com/3545/4784804/review...,6.42015E+11,14 pounds,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[['name','brand','categories']]
df.head()

,name,brand,categories
0,Sanus VLF410B1 10-Inch Super Slim Full-Motion ...,Sanus,"Audio & Video Accessories,TV Mounts,TV Accesso..."
1,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ..."
2,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ..."
3,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ..."
4,Boytone - 2500W 2.1-Ch. Home Theater System - ...,Boytone,"Stereos,Portable Bluetooth Speakers,TV, Video ..."


In [4]:
df['name'] = df['name'].str.lower().replace(['^a-zA-Z0-9']," ",regex=True) 
df['categories'] = df['categories'].str.lower().replace(['^a-zA-Z0-9']," ",regex=True) 

In [5]:
df.head()

,name,brand,categories
0,sanus vlf410b1 10-inch super slim full-motion ...,Sanus,"audio & video accessories,tv mounts,tv accesso..."
1,boytone - 2500w 2.1-ch. home theater system - ...,Boytone,"stereos,portable bluetooth speakers,tv, video ..."
2,boytone - 2500w 2.1-ch. home theater system - ...,Boytone,"stereos,portable bluetooth speakers,tv, video ..."
3,boytone - 2500w 2.1-ch. home theater system - ...,Boytone,"stereos,portable bluetooth speakers,tv, video ..."
4,boytone - 2500w 2.1-ch. home theater system - ...,Boytone,"stereos,portable bluetooth speakers,tv, video ..."


In [6]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
stemr = SnowballStemmer('english')

In [7]:
def tokenization(txt):
    tokens = nltk.word_tokenize(txt.lower())
    stemming = [stemr.stem(w) for w in tokens if not w in stopwords.words('english') and w.isalnum()]
    return " ".join(stemming)

In [8]:
df['name'] = df['name'].apply(lambda x: tokenization(x))
df['categories'] = df['categories'].apply(lambda x: tokenization(x))

In [9]:
df['name_cate'] = df['name'] + " " + df['categories']

In [10]:
df['name_cate'][0]

'sanus vlf410b1 super slim mount 37 84 inch tv audio video accessori tv mount tv accessori part electron present accessori suppli tv ceil wall mount'

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [12]:
def cosine_sim(txt1,txt2):
    obj_tfidf = TfidfVectorizer(tokenizer=tokenization)
    tfidfmatrix = obj_tfidf.fit_transform([txt1,txt2])
    similarity = cosine_similarity(tfidfmatrix)[0][1]
    return similarity

In [13]:
def recommation(query):
    tokenized_query = tokenization(query)
    df['similarity'] = df['name_cate'].apply(lambda x: cosine_sim(tokenized_query,x))
    final_df = df.sort_values(by=['similarity'],ascending=False).head(20)[['name','brand','categories','sourceURLs']]
    return final_df